### CoinMarketCap Study
- API documentation is enable to be read in https://coinmarketcap.com/api/documentation/v1/#

### Health-Check

In [4]:
import requests as r
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import os
import pandas as pd

- Testing the API connection in the sandbox environment

In [5]:
test_url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'b54bcf4d-1bca-4e8e-9a24-22ff2c3d462c',
}

session = r.Session()
session.headers.update(headers)

try:
  response = session.get(test_url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'timestamp': '2024-12-27T21:44:58.519Z', 'error_code': 0, 'error_message': None, 'elapsed': 1, 'credit_count': 1, 'notice': None}, 'data': [{'id': 9939, 'name': 'jksqb4fkrbg', 'symbol': 'dqm2ltyiq8c', 'slug': 'jidjcw78a1', 'cmc_rank': 3005, 'num_market_pairs': 731, 'circulating_supply': 4532, 'total_supply': 3382, 'max_supply': 4373, 'infinite_supply': None, 'last_updated': '2024-12-27T21:44:58.519Z', 'date_added': '2024-12-27T21:44:58.519Z', 'tags': ['ppxh64jcjs', '4r9mxvsv1co', 'awqm8yvu6sn', 'qhvm2n4hdd', 'h4rrgi09sxl', 'xwzregnh7h', 'sl6y1x9v72', 'pmng6b1ndu', 'ffs0eqzvt3a', '1mfiz1a0hjn'], 'platform': None, 'self_reported_circulating_supply': None, 'self_reported_market_cap': None, 'quote': {'USD': {'price': 0.660685993441257, 'volume_24h': 3208, 'volume_change_24h': 0.5902902391452838, 'percent_change_1h': 0.3773920115147491, 'percent_change_24h': 0.43601007392795643, 'percent_change_7d': 0.2916652594641933, 'market_cap': 0.5681027804468033, 'market_cap_dominance': 46

- Testing the API connection in production environment with my own user credentials

In [6]:
# defining the base url that will be useful on any later calls
base_url= os.environ.get('COIN_MARKET_API_BASE_URL')

# defining the access_token regarding my own user in the subscribed platform
access_token= os.environ.get('COIN_MARKET_API_ACCESS_TOKEN')

# api call parameters (here using the same as above)
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}

# redefining the headers variable according to documentation reference, but with the appropriate pointer to my user access token
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': access_token,
}

# doing the api call attempt
test_url= base_url+'/v1/cryptocurrency/listings/latest'
try:
    response = r.get(test_url, params= parameters, headers= headers)
    data= json.loads(response.text)
    #print(data) 
    print(response.status_code)
    print(response.reason)
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)


200
OK


In [7]:
data

{'status': {'timestamp': '2024-12-27T21:45:23.228Z',
  'error_code': 0,
  'error_message': None,
  'elapsed': 81,
  'credit_count': 25,
  'notice': None,
  'total_count': 10483},
 'data': [{'id': 1,
   'name': 'Bitcoin',
   'symbol': 'BTC',
   'slug': 'bitcoin',
   'num_market_pairs': 11853,
   'date_added': '2010-07-13T00:00:00.000Z',
   'tags': ['mineable',
    'pow',
    'sha-256',
    'store-of-value',
    'state-channel',
    'coinbase-ventures-portfolio',
    'three-arrows-capital-portfolio',
    'polychain-capital-portfolio',
    'binance-labs-portfolio',
    'blockchain-capital-portfolio',
    'boostvc-portfolio',
    'cms-holdings-portfolio',
    'dcg-portfolio',
    'dragonfly-capital-portfolio',
    'electric-capital-portfolio',
    'fabric-ventures-portfolio',
    'framework-ventures-portfolio',
    'galaxy-digital-portfolio',
    'huobi-capital-portfolio',
    'alameda-research-portfolio',
    'a16z-portfolio',
    '1confirmation-portfolio',
    'winklevoss-capital-portfol

### Exploring API endpoints and exploring what can be useful

#### 1) Cryptocurrency metadata
- According to documentation, we could be doing the API call using parameters *slug* or *symbol*, but symbol would return every cryptocurrency associated to the value passed (for example, by searching for *BTC* we would find other cryptocurrencies beyond the usual bitcoin) 

In [8]:
# defining the full query URL
metadata_url= base_url+'/v2/cryptocurrency/info'

# defining the appropriate parameters (I want to see basic information involving some of the main recent cryptocurrencies)
parameters= {
    'symbol': 'BTC,ETH,USDT,USDC,BNB,XRP,ADA,BUSD,SOL,DOT'
}
# Here we are using the symbol parameter in order to include 'Binance USD (BUSD)', 'Binance Coin (BNB)' and 'USD Coin (USDC)' in the search since their names have spaces and we could not specify them on slug argument. 

# But by doing it is important to emphasize a thing: using symbol parameter, the API will return to us any reference it has for such symbol stored in the endpoint we are calling (and it might not necessarily be a crypto). So we'll have to filter it later on

# making the call attempt
response= r.get(
    url= metadata_url
    ,params= parameters
    ,headers= headers
)

print(response.status_code)
print(response.reason)

# printing the entire response to take a look at the error message if there's a HTTP expection of type 4xx (i.e., user inconsistency)
if str(response.status_code)[0] == '4':
    print(response.json()) 

200
OK


In [10]:
# special treatment to handle all returned data and compile it on a single dataframe
crypto_symbols= ['BTC','ETH','USDT','USDC','BNB','XRP','ADA','BUSD','SOL','DOT']
df_data= []

for crypto in crypto_symbols:
    aux_df= pd.json_normalize(response.json()['data'][crypto])
    df_data.append(aux_df)

df= pd.concat(df_data, ignore_index= True)
df.head()

C:\Users\LUCAS\AppData\Local\Temp\ipykernel_18952\3920442243.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df= pd.concat(df_data, ignore_index= True)


,id,name,symbol,category,description,slug,logo,subreddit,notice,tags,...,urls.explorer,urls.reddit,urls.technical_doc,urls.source_code,urls.announcement,platform.id,platform.name,platform.slug,platform.symbol,platform.token_address
0,1,Bitcoin,BTC,coin,Bitcoin (BTC) is a cryptocurrency launched in ...,bitcoin,https://s2.coinmarketcap.com/static/img/coins/...,bitcoin,,"[mineable, pow, sha-256, store-of-value, state...",...,"[https://blockchain.info/, https://live.blockc...",[https://reddit.com/r/bitcoin],[https://bitcoin.org/bitcoin.pdf],[https://github.com/bitcoin/bitcoin],[],NaN,NaN,NaN,NaN,NaN
1,34316,HarryPotterTrumpSonic100Inu,BTC,token,HarryPotterTrumpSonic100Inu (BTC) is a cryptoc...,harrypottertrumpsonic100inu,https://s2.coinmarketcap.com/static/img/coins/...,,,[memes],...,[https://etherscan.io/token/0x7099ab9e42fa7327...,[],[],[],[],1027,Ethereum,ethereum,ETH,0x7099aB9E42Fa7327a6b15E0a0c120c3e50d11BeC
2,31652,batcat,BTC,token,batcat (BTC) is a cryptocurrency launched in 2...,batcat,https://s2.coinmarketcap.com/static/img/coins/...,,,"[memes, solana-ecosystem]",...,[https://solscan.io/token/EtBc6gkCvsB9c6f5wSbw...,[],[],[],[],5426,Solana,solana,SOL,EtBc6gkCvsB9c6f5wSbwG8wPjRqXMB5euptK6bqG1R4X
3,30938,Satoshi Pumpomoto,BTC,token,Satoshi Pumpomoto (BTC) is a cryptocurrency la...,satoshi-pumpomoto,https://s2.coinmarketcap.com/static/img/coins/...,,,None,...,[https://solscan.io/token/6AGNtEgBE2jph1bWFdya...,[],[],[],[],5426,Solana,solana,SOL,6AGNtEgBE2jph1bWFdyaqsXJ762emaP9RE17kKxEsfiV
4,31469,Boost Trump Campaign,BTC,token,Boost Trump Campaign (BTC) is a cryptocurrency...,boost-trump-campaign,https://s2.coinmarketcap.com/static/img/coins/...,,,None,...,[https://etherscan.io/token/0x300e0d87f8c95d90...,[],[],[],[],1027,Ethereum,ethereum,ETH,0x300e0d87f8c95d90cfe4b809baa3a6c90e83b850


In [11]:
df['category'].unique() # checking different categories of the extracted data

array(['coin', 'token'], dtype=object)

- As we previously mentioned, by filtering the query with the *symbol* argument we allow it to return to us different results that are not exclusively related to cryptos. We have te filter that

In [12]:
df= df[df['category'] == 'coin'] # applying filter on DataFrame

In [13]:
df['category'].unique() # checking the result

array(['coin'], dtype=object)

In [14]:
df.isna().sum() # verifying the quantity of null values inside the dataframe

id                                  0
name                                0
symbol                              0
category                            0
description                         0
slug                                0
logo                                0
subreddit                           0
notice                              0
tags                                0
tag-names                           0
tag-groups                          0
platform                            8
date_added                          0
twitter_username                    0
is_hidden                           0
date_launched                       5
contract_address                    0
self_reported_circulating_supply    8
self_reported_tags                  6
self_reported_market_cap            8
infinite_supply                     0
urls.website                        0
urls.twitter                        0
urls.message_board                  0
urls.chat                           0
urls.faceboo

In [16]:
# saving a final csv file
df.to_csv('API-data/cryptocurrency_metada.csv', sep= ';', index= False)

#### 2) Cryptocurrency quotes

#### 3) Cryptocurrency market